In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Input, Dense
from keras.models import Model
from keras.layers import Flatten
from keras.layers import merge
from keras.layers import Reshape
from keras.layers import Activation
from keras.layers import concatenate

C:\Users\Ryoji Nomura\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# define IO shapes
input_shape = (300, 300, 3)
num_classes = 3

In [3]:
## root model
net = {}

# Input
net['inputs'] = Input(shape=input_shape, name='input')

# Block 1
net['conv1_1'] = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv1_1',input_shape=input_shape)(net['inputs'])
net['conv1_2'] = Conv2D(64, (3, 3),activation='relu',padding='same',name='conv1_2')(net['conv1_1'])
net['pool1'] = MaxPooling2D((2,2),strides=(2,2),padding='same',name='pool1')(net['conv1_2'])

# # Block 2
net['conv2_1'] = Conv2D(128, (3, 3),activation='relu',padding='same',name='conv2_1')(net['pool1'])
net['conv2_2'] = Conv2D(128, (3, 3),activation='relu',padding='same',name='conv2_2')(net['conv2_1'])
net['pool2'] = MaxPooling2D((2, 2), strides=(2, 2), padding='same',name='pool2')(net['conv2_2'])

# # FC3
net['fc3'] = Conv2D(1024, (3, 3), dilation_rate=(6, 6),activation='relu', padding='same',name='fc3')(net['pool2'])

# # FC4
net['fc4'] = Conv2D(1024, (1, 1), activation='relu',padding='same',name='fc4')(net['fc3'])

# # Block 5
net['conv5_1'] = Conv2D(128, (3, 3),activation='relu',padding='same',name='conv5_1')(net['fc4'])
net['pool5'] = MaxPooling2D((2, 2), strides=(2, 2), padding='same',name='pool5')(net['conv5_1'])

# # Block 6
net['conv6_1'] = Conv2D(128, (3, 3),activation='relu',padding='same',name='conv6_1')(net['pool5'])
net['pool6'] = MaxPooling2D((2, 2), strides=(2, 2), padding='same',name='pool6')(net['conv6_1'])

# # Last Pool
net['pool7'] = GlobalAveragePooling2D(name='pool7')(net['pool6'])
net['dence8'] = Dense(num_classes, activation='softmax', name='dence8')(net['pool7'] )

model = Model(input=net['inputs'], output=net['dence8'])

C:\Users\Ryoji Nomura\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 300, 300, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 300, 300, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 300, 300, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 150, 150, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 150, 150, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 150, 150, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 75, 75, 128)       0         
__________

In [5]:
## branch model for feature map

dim_box = 4

num_priors = 6
net['conv5_1_mbox_loc'] = Conv2D(num_priors*dim_box,(3,3),padding='same',name='conv5_1_mbox_loc')(net['conv5_1'])
net['conv5_1_mbox_loc_flat'] = Flatten(name='conv5_1_mbox_loc_flat')(net['conv5_1_mbox_loc'])

net['conv5_1_mbox_conf'] = Conv2D(num_priors*num_classes,(3,3),padding='same',name='conv5_1_mbox_conf')(net['conv5_1'])
net['conv5_1_mbox_conf_flat'] = Flatten(name='conv5_1_mbox_conf_flat')(net['conv5_1_mbox_conf'])

net['pool6_mbox_loc'] = Conv2D(num_priors*dim_box,(3,3),padding='same',name='pool6_mbox_loc')(net['pool6'])
net['pool6_mbox_loc_flat'] = Flatten(name='pool6_mbox_loc_flat')(net['pool6_mbox_loc'])

net['pool6_mbox_conf'] = Conv2D(num_priors*num_classes,(3,3),padding='same',name='pool6_mbox_conf')(net['pool6'])
net['pool6_mbox_conf_flat'] = Flatten(name='pool6_mbox_conf_flat')(net['pool6_mbox_conf'])

# priorbox = PriorBox(img_size, 60.0, max_size=114.0, aspect_ratios=[2, 3],
#                     variances=[0.1, 0.1, 0.2, 0.2],
#                     name='conv5_1_mbox_priorbox')

# net['conv5_1_mbox_priorbox'] = priorbox(net['conv5_1'])

net['mbox_loc'] = concatenate([net['conv5_1_mbox_loc_flat'], net['pool6_mbox_loc_flat']], name='mbox_loc')
net['mbox_conf'] = concatenate([net['conv5_1_mbox_conf_flat'],net['pool6_mbox_conf_flat']],name='mbox_conf')
# net['mbox_conf'] = concatenate([net['conv5_1_mbox_priorbox'],net['pool6_mbox_priorbox']],name='mbox_conf')

net['mbox_loc'] = Reshape((-1, dim_box),name='mbox_loc_final')(net['mbox_loc'])
net['mbox_conf'] = Reshape((-1, num_classes),name='mbox_conf_logits')(net['mbox_conf'])
net['mbox_conf'] = Activation('softmax',name='mbox_conf_final')(net['mbox_conf'])
net['predictions'] = concatenate([net['mbox_loc'], net['mbox_conf']], name='predictions')

# net['predictions'] = concatenate([net['mbox_loc'], net['mbox_conf'], net['mbox_priorbox']], name='predictions')

model_feature_map = Model(input=net['inputs'], output=net['predictions'])

C:\Users\Ryoji Nomura\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`


In [6]:
model_feature_map.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 300, 300, 64) 1792        input[0][0]                      
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, 300, 300, 64) 36928       conv1_1[0][0]                    
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 150, 150, 64) 0           conv1_2[0][0]                    
__________________________________________________________________________________________________
conv2_1 (C

In [7]:
import keras.backend as K
K.int_shape(net['pool6_mbox_conf'])

(None, 19, 19, 18)

In [8]:
from ssd_layer import DefaultBox

In [9]:
boxes = DefaultBox(input_shape, 20, 30, [1], 0.1)
boxes(net['pool6_mbox_conf'])

<tf.Tensor 'default_box_1/ExpandDims:0' shape=(1, 361, 8) dtype=float32>

In [11]:
defaultbox = DefaultBox(input_shape, 20, 30, aspect_ratios=[2, 3],
                    variance=[0.1],
                    name='conv5_1_mbox_priorbox')

net['conv5_1_mbox_priorbox'] = defaultbox(net['conv5_1'])

In [12]:
net['conv5_1_mbox_priorbox']

<tf.Tensor 'conv5_1_mbox_priorbox/ExpandDims:0' shape=(1, 11250, 8) dtype=float32>

In [13]:
net['conv5_1']

<tf.Tensor 'conv5_1/Relu:0' shape=(?, 75, 75, 128) dtype=float32>